In [1]:
#1 读取Excel文件

import pandas
import math

filename = "原始H.xlsx"
path = "D:\\熵权\\"
data = pandas.read_excel(path+filename,sheet_name=0)
#保留原始数据
data_yuanshi = data.copy()
print(data)


                 漏斗  地灾       经济损         高程         降水         温差
0     181015.865792   0  357436.0  30.113244  21.390701  33.582880
1      64755.509011   0  357436.0  24.416763  22.191440  33.702040
2      17572.321407   0  357436.0  26.415722  21.741275  34.076674
3          0.000000   0  186154.0  45.854378  22.654401  33.264549
4       7364.964501   0  357436.0  49.383379  22.140590  33.415480
...             ...  ..       ...        ...        ...        ...
1090    2352.078201   0  186154.0  41.212604  23.140989  32.302056
1091       0.000000   2  186154.0  57.027415  23.377501  32.335350
1092   89552.556152   0  186154.0  54.438883  23.567540  32.338740
1093   35353.016393   0  186154.0  44.420131  22.932312  32.521276
1094       0.000000   0  186154.0  55.781662  23.166150  32.222549

[1095 rows x 6 columns]


In [2]:
#2 归一化化数据

max = data.max()
min = data.min()
cols = data.columns
#归一化数据
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        data.iloc[i,j] = (float(data.iloc[i,j])-float(min[cols[j]]))/(float(max[cols[j]])-float(min[cols[j]]))
data_guiyi = data.copy()
print(data)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_12880\2739837167.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.3333333333333333' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.iloc[i,j] = (float(data.iloc[i,j])-float(min[cols[j]]))/(float(max[cols[j]])-float(min[cols[j]]))


            漏斗        地灾       经济损        高程        降水        温差
0     0.094440  0.000000  0.745134  0.363139  0.359653  0.534844
1     0.033785  0.000000  0.745134  0.292844  0.535524  0.579469
2     0.009168  0.000000  0.745134  0.317511  0.436652  0.719767
3     0.000000  0.000000  0.382346  0.557387  0.637206  0.415631
4     0.003842  0.000000  0.745134  0.600936  0.524355  0.472154
...        ...       ...       ...       ...       ...       ...
1090  0.001227  0.000000  0.382346  0.500107  0.744078  0.055184
1091  0.000000  0.666667  0.382346  0.695264  0.796025  0.067652
1092  0.046722  0.000000  0.382346  0.663321  0.837764  0.068922
1093  0.018445  0.000000  0.382346  0.539688  0.698246  0.137280
1094  0.000000  0.000000  0.382346  0.679891  0.749605  0.025409

[1095 rows x 6 columns]


In [3]:
#3 计算每条记录的信息熵

#存储每一条记录的熵
result_sum_col = []
for j in range(data.shape[1]):
    sum_col = data[cols[j]].sum()
    # print(sum_col)
    for i in range(data.shape[0]):
        if data.iloc[i,j] == 0:
            #计算Log(0)是会报错，需要特殊处理
            data.iloc[i, j] = 0
        else:
            data.iloc[i,j] = -1*(data.iloc[i,j]/sum_col)*math.log10((data.iloc[i,j]/sum_col))
    #该指标的熵等于各记录熵的和除以Log(n)，n为记录的条数
    result_sum_col.append(data[cols[j]].sum()/math.log10(data.shape[0]))
    # print(result_sum_col)
print(data)


            漏斗        地灾       经济损        高程        降水        温差
0     0.020013  0.000000  0.003678  0.002407  0.002034  0.003093
1     0.008758  0.000000  0.003678  0.001999  0.002865  0.003312
2     0.002927  0.000000  0.003678  0.002144  0.002404  0.003982
3     0.000000  0.000000  0.002076  0.003473  0.003324  0.002492
4     0.001381  0.000000  0.003678  0.003703  0.002813  0.002780
...        ...       ...       ...       ...       ...       ...
1090  0.000505  0.000000  0.002076  0.003166  0.003792  0.000425
1091  0.000000  0.022596  0.002076  0.004190  0.004016  0.000509
1092  0.011414  0.000000  0.002076  0.004026  0.004193  0.000517
1093  0.005295  0.000000  0.002076  0.003379  0.003593  0.000951
1094  0.000000  0.000000  0.002076  0.004111  0.003816  0.000212

[1095 rows x 6 columns]


In [4]:
#4 计算各指标的权重

weight = []
weight_sum = 0.0
for i in result_sum_col:
    weight_sum += i
#'''
#    i表示各个维度的熵,n表示维度的个数
#    各维度计算公式为（1-i）/(n-sum_i)
#'''
for i in result_sum_col:
    weight.append((1-i)/(float(data.shape[1])*1-weight_sum))
print(weight)


[0.4556611182364941, 0.4187812861819938, 0.03007944921953087, 0.033256966629297974, 0.018652897993528993, 0.043568281739154585]


In [5]:
#5 计算每条记录最终得分

#计算各记录最终得分并另存文件(原始数据文件)
cores = []
for i in range(data_yuanshi.shape[0]):
    result_core = 0
    for j in range(data_yuanshi.shape[1]):
        result_core += float(data_yuanshi.iloc[i,j])*weight[j]
    cores.append(result_core)
print(cores)
data_yuanshi["最终评分"] = cores
print(data_yuanshi)
#计算各记录最终得分并另存文件(归一化数据文件)
cores2 = []
for i in range(data_guiyi.shape[0]):
    result_core = 0
    for j in range(data_guiyi.shape[1]):
        result_core += float(data_guiyi.iloc[i,j])*weight[j]
    cores2.append(result_core)
print(cores2)
data_guiyi["最终评分"] = cores2
print(data_guiyi)


[93236.23345865397, 40260.739961126244, 18761.27034029035, 5602.806617009961, 14110.917154184865, 17459.000626410892, 5603.264272557257, 10754.864912139326, 10755.092726094326, 5603.687619734491, 10755.855835881795, 173.16731709783915, 10753.558869407869, 12880.698561478184, 5603.046051568859, 10844.744550640076, 20977.035177301914, 5602.80202863061, 11466.743053193373, 172.5698049633628, 11205.92921187392, 10753.835479226593, 10754.457615589896, 56515.04322261952, 5602.979701207519, 10755.21300495769, 10754.953536238725, 10755.10363065876, 10753.649897237994, 10754.303950095324, 10755.706851593208, 12650.806993378112, 10799.12096187851, 5602.939455185526, 5603.0971816755, 16101.27442062009, 107296.15861037586, 10754.329918231513, 6945.509749766365, 10753.570034919112, 10753.907821335311, 10755.64989594099, 12868.529652494659, 896.7914561397181, 5603.051201569096, 10753.683468690047, 10755.147483167451, 12153.071553894575, 5603.120888936256, 47267.07635018167, 35859.234267329084, 11839

In [6]:
#6 保存文件

#'''
#    熵文件中不用计算最终得分，但要写入指标权重
#'''
data.loc["权重"] = weight
print(data)
#保存文件
file01 = filename.split(".")[0]+"最终得分.xlsx"
file02 = filename.split(".")[0]+"归一化.xlsx"
file03 = filename.split(".")[0]+"熵.xlsx"
data_yuanshi.to_excel(path+file01,index=False)
data_guiyi.to_excel(path+file02,index=False)
data.to_excel(path+file03)


            漏斗        地灾       经济损        高程        降水        温差
0     0.020013  0.000000  0.003678  0.002407  0.002034  0.003093
1     0.008758  0.000000  0.003678  0.001999  0.002865  0.003312
2     0.002927  0.000000  0.003678  0.002144  0.002404  0.003982
3     0.000000  0.000000  0.002076  0.003473  0.003324  0.002492
4     0.001381  0.000000  0.003678  0.003703  0.002813  0.002780
...        ...       ...       ...       ...       ...       ...
1091  0.000000  0.022596  0.002076  0.004190  0.004016  0.000509
1092  0.011414  0.000000  0.002076  0.004026  0.004193  0.000517
1093  0.005295  0.000000  0.002076  0.003379  0.003593  0.000951
1094  0.000000  0.000000  0.002076  0.004111  0.003816  0.000212
权重    0.455661  0.418781  0.030079  0.033257  0.018653  0.043568

[1096 rows x 6 columns]
